# Temp table

## Original comments

see https://github.com/ibis-project/ibis/issues/2339

marianoteehan commented on Aug 18

>
> A common pattern for me is to build up a SQL expression, execute it against the database backend and save the results as a database table. I think it's not currently possible to do this entirely in Ibis? What I generally do is compile the expression to SQL and then execute that using either the BigQuery web interface or the Python client, both of which have options to save the query as a table. It would be really nice if I could do all of this without leaving Ibis.
> 
> Some other things I often do:
> 
> List datasets (Ibis does this)
> List tables (Ibis does this)
> Create and delete tables and datasets
> Execute queries and save the results directly as files on GCS or local disk
> Copy tables to local disk or Google Cloud Storage, e.g. as CSV, Avro, etc
> Upload CSV files or dataframes to database tables
> I realize this stuff is potentially out of scope, but just thought I'd ask. So far I end up writing a wrapper library around Ibis to do all of this but it strikes me that it could fit really nicely in the main Ibis project. I think some of the Blaze project was focused on use cases like this and I > think maybe there's a gap here.


niviksha commented on Aug 26

>
> this is an interesting request - I'd asked for something similar where any expr.execute could support an as parameter, which would leverage the underlying engine's support create table (or even the select ...INTO variant supported by sql server and the like)
> 
> so something like
> 
> expr.execute(as='newTable', temp=True)
> 
> Bumping this up - It would be great to add the ability to do this esp for OmniSciDB but any backend that supports temporary tables( cc @xmnlab ) - one common pattern I have to run an ibis expression and use that materialized result for the next set of Ibis expressions.
> 
> The big advantage of this, is that it allows potentially large intermediate result sets not to have to return client-side, esp if temp tables are supported by the backend - i.e the materialization is controllable by the user via this option
> 
> e.g. something like
> 
> expr1 = t.groupby(t.a).aggregate(t.count().name('ct'))
result = expr1.execute(as='newTable', temp=True) #this would return a TableExpr wrapping 'newTable'
result.filter(ct > 500) #...etc
> 
> #we could even chain this
> result = expr1.execute(as='newTable', temp=True).filter(ct>500)

OmniSci temporary table

CREATE [TEMPORARY] TABLE [IF NOT EXISTS] <table>
  (<column> <type> [NOT NULL] [DEFAULT <value>] [ENCODING <encodingSpec>],
  [SHARD KEY (<column>)],
  [SHARED DICTIONARY (<column>) REFERENCES <table>(<column>)], ...)
  [WITH (<property> = value, ...)]
    
    

In [1]:
# https://github.com/ibis-project/ibis/pull/2112
from typing import Dict, Optional
import ibis
import pandas as pd

In [2]:
db_conf = dict(
    host='localhost', port='6274', 
    user='admin', password='HyperInteractive', 
    database='omnisci'
)
con = ibis.omniscidb.connect(**db_conf)

con.list_tables()

['omnisci_states',
 'omnisci_counties',
 'omnisci_countries',
 'omnisci_states_tmp']

In [3]:
help(con.create_table)

Help on method create_table in module ibis.backends.omniscidb.client:

create_table(table_name: str, obj: Union[ibis.expr.types.TableExpr, pandas.core.frame.DataFrame, NoneType] = None, schema: Union[ibis.expr.schema.Schema, NoneType] = None, database: Union[str, NoneType] = None, max_rows: Union[int, NoneType] = None, fragment_size: Union[int, NoneType] = None, is_temporary: bool = False, **kwargs) method of ibis.backends.omniscidb.client.OmniSciDBClient instance
    Create a new table from an Ibis table expression.
    
    Parameters
    ----------
    table_name : string
    obj : ibis.expr.types.TableExpr or pandas.DataFrame, optional
      If passed, creates table from select statement results
    schema : ibis.Schema, optional
    table_name : str
    obj : TableExpr or pandas.DataFrame, optional, default None
      If passed, creates table from select statement results.
    schema : ibis.Schema, optional, default None
      Mutually exclusive with expr, creates an empty table w

In [4]:
table_name = 'omnisci_states'
cols = ['id', 'abbr', 'name']

expr = con.table(table_name)[cols]

new_table_name = f'{table_name}_tmp'
con.create_table(new_table_name, expr, is_temporary=True)

con.table(new_table_name).execute()

CREATE TABLE omnisci_states_tmp
AS (
SELECT "id", "abbr", "name"
FROM omnisci_states
)


Exception: 'NoneType' object has no attribute 'strip': None

In [ ]:
con.table(new_table_name).execute()